# Feature Selection II - Selecting for Model Accuracy

In this second chapter on feature selection, you'll learn how to let models help you find the most important features in a dataset for predicting a particular target feature. In the final lesson of this chapter, you'll combine the advice of multiple, different, models to decide on which features are worth keeping.

## Selecting features for model performance

So far we've only considered individual or pairwise properties of features to decide whether we keep them in the dataset or not.

Another approach is to select features based on how they affect model performance. 

### Ansur dataset sample

Consider this sample of the ANSUR dataset with one target variable, "Gender" which we'll try to predict, and five body measurement features to do so. 

| Gender | chestdepth | handlength | neckcircumference | shoulderlength | earlength |
|--------|-----------:|-----------:|------------------:|---------------:|----------:|
| Female | 243 | 176 | 326 | 136 | 62 |
| Female | 219 | 177 | 325 | 135 | 58 |
| Male | 259 | 193 | 400 | 145 | 71 |
| Male | 253 | 195 | 380 | 141 | 62 |

### Pre-processing the data

To train a model on this data we should first perform a train-test-split, and in this case also standardize the training feature dataset `X_train` to have a mean of zero and a variance of one. Notice that we've used the `.fit_transform()` method to fit the scaler and transform the data in one command.

``` python
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, t_test = train_test_split(X, y, test_size=0.3)

scaler = StandardScaler()

X_train_std = scaler.fit_tramsform(X_train)
```

### Creating a logistic regression model

We can then create and fit a logistic regression model on this standardized training data. To see how the model performs on the test set we first scale these features with the `.transform()` method of the scaler that we just fit on the training set and then make our prediction. we get a test-set accuracy of 99%. 

```python
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr = LogisticRegression()
lr.fit(X_train_std, y_train)

X_test_std = scaler.transform(X_test)

y_pred = lr.predict(X_test_std)

print(accuracy_score(y_test, y_pred))
```

```sh
0.99
```

### Inspecting the feature coefficients

However, when we look at the feature coefficients that the logistic regression model uses in its decision function, we'll see that some values are pretty close to zero.

```python
print(lr.coef_)
```

```sh
array([[-3. , 0.14, 7.46, 1.22, 0.87]])
```

Since these coefficients will be multiplied with the feature values when the model makes a prediction, features with coefficients close to zero will contribute little to the end result.

We can use the zip function to transform the output into a dictionary that shows which feature has which coefficient.

```python
print(dict(zip(X.columns, abs(lr.coef_[0]))))
```

```sh
{'chestdepth': 3.0,
 'handlength': 0.14,
 'neckcircumference': 7.46,
 'shoulderlength': 1.22,
 'earlength': 0.87}
```

If we want to remove a feature from the initial dataset with as little effect on the predictions as possible, we should pick the one with the lowest coefficient, "handlength" in this case. **The fact that we standardized the data first makes sure that we can compare the coefficients to one another.**

### Features that contribute little to a model

When we remove the "handlength" feature at the start of the process, our model accuracy remains unchanged at 99% while we did reduce our dataset's complexity.

```python
X.drop('handlength', axes=1, inplace=True)

X_train, X_test, y_train, t_test = train_test_split(X, y, test_size=0.3)

lr.fit(scaler.fit_transform(X_train), y_train)

print(accuracy_score(y_test, lr.predict(scaler.transform(X_test))))
```

```sh
0.99
```

We could repeat this process until we have the desired number of features remaining, but it turns out, there's a Scikit-learn function that does just that. 

### Recursive Feature Elimination

Recursive Feature Elimination (RFE) is a feature selection algorithm that can be wrapped around any model that produces feature coefficients or feature importance values. We can pass it the model we want to use and the number of features we want to select. While fitting to our data it will repeat a process where it first fits the internal model and then drops the feature with the weakest coefficient. It will keep doing this until the desired number of features is reached. If we set RFE's verbose parameter higher than zero we'll be able to see that features are dropped one by one while fitting. We could also decide to just keep the 2 features with the highest coefficients after fitting the model once, but this recursive method is safer, since **dropping one feature will cause the other coefficients to change**. 

```python
from sklearn.feature_selection import RFE

rfe = RFE(estimator=LogisticRegression(), n_features_to_select=2, verbose=1)
rfe.fit(X_train_std, y_train)
```

```sh
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
```

### Inspecting the RFE results

Once RFE is done we can check the `support_` attribute that contains `True`/`False` values to see which features were kept in the dataset.

```python
X.columns[rfe.support_]
```

```sh
Index(['chestdepth', 'neckcircumference'], dtype='object')
```

Using the `zip` function once more, we can also check out rfe's `ranking_` attribute to see in which iteration a feature was dropped. Values of 1 mean that the feature was kept in the dataset until the end while high values mean the feature was dropped early on.

```python
print(dict(zip(X.columns, rfe.ranking_)))
```

```sh
{'chestdepth': 1,
 'handlength': 4,
 'neckcircumference': 1,
 'shoulderlength': 2,
 'earlength': 3}
```

Finally, we can test the accuracy of the model with just two remaining features, `'chestdepth'` and `'neckcircumference'`, turns out the accuracy is still untouched at 99%. This means the other 3 features had little to no impact on the model an its predictions. 

```python
print(accuracy_score(y_test, rfe.predict(X_test_std)))
```

```sh
0.99
```

#### Building a diabetes classifier

You'll be using the Pima Indians diabetes dataset to predict whether a person has diabetes using logistic regression. There are 8 features and one target in this dataset. The data has been split into a training and test set and pre-loaded for you as `X_train`, `y_train`, `X_test`, and `y_test`.

A `StandardScaler()` instance has been predefined as `scaler` and a `LogisticRegression()` one as `lr`.

##### Instructions

* Fit the scaler on the training features and transform these features in one go.
* Fit the logistic regression model on the scaled training data.
* Scale the test features.
* Predict diabetes presence on the scaled test set.

In [ ]:
# setup
import pandas as pd

from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

diabetes_df = pd.read_csv("data/pima_indians_diabetes_2.csv")

y = diabetes_df["test"]
X = diabetes_df.drop("test", axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

scaler = StandardScaler()
lr = LogisticRegression()

In [ ]:
# fit the scaler on the training features and transform in one go
X_train_std = scaler.fit_transform(X_train)

# fit the logistic regression model on the scaled training data
lr.fit(X_train_std, y_train)

# scale the test features
X_test_std = scaler.transform(X_test)

# predict diabetes presence on the scaled test set
y_pred = lr.predict(X_test_std)

# Prints accuracy metrics and feature coefficients
print(f"{accuracy_score(y_test, y_pred):.1%} accuracy on test set.")
print(dict(zip(X.columns, abs(lr.coef_[0]).round(2))))

#### Manual recursive feature elimination

Now that we've created a diabetes classifier, let's see if we can reduce the number of features without hurting the model accuracy too much.

On the second line of code the features are selected from the original DataFrame. Adjust this selection.

A `StandardScaler()` instance has been predefined as `scaler` and a `LogisticRegression()` one as `lr`.

All necessary functions and packages have been pre-loaded too.

##### Instructions 1/3

* First, run the given code, then remove the feature with the lowest model coefficient from X.

In [ ]:
# remove the feature with the lowest model coefficient
# X = diabetes_df[["pregnant", "glucose", "diastolic", "triceps", "insulin", "bmi", "family", "age"]]
X = diabetes_df[["pregnant", "glucose", "triceps", "insulin", "bmi", "family", "age"]]

# performs a 25-75% train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

# scales features and fits the logistic regression model
lr.fit(scaler.fit_transform(X_train), y_train)

# calculates the accuracy on the test set and prints coefficients
acc = accuracy_score(y_test, lr.predict(scaler.transform(X_test)))
print(f"{acc:.1%} accuracy on test set.")
print(dict(zip(X.columns, abs(lr.coef_[0]).round(2))))

##### Instructions 2/3

* Run the code and remove 2 more features with the lowest model coefficients.

In [ ]:
# Remove the 2 features with the lowest model coefficients
# X = diabetes_df[["pregnant", "glucose", "triceps", "insulin", "bmi", "family", "age"]]
X = diabetes_df[["glucose", "triceps", "bmi", "family", "age"]]

# Performs a 25-75% train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

# Scales features and fits the logistic regression model
lr.fit(scaler.fit_transform(X_train), y_train)

# Calculates the accuracy on the test set and prints coefficients
acc = accuracy_score(y_test, lr.predict(scaler.transform(X_test)))
print(f"{acc:.1%} accuracy on test set.")
print(dict(zip(X.columns, abs(lr.coef_[0]).round(2))))

##### Instructions 3/3

* Run the code and only keep the feature with the highest coefficient.

In [ ]:
# Only keep the feature with the highest coefficient
# X = diabetes_df[["glucose", "triceps", "bmi", "family", "age"]]
X = diabetes_df[["glucose"]]

# Performs a 25-75% train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

# Scales features and fits the logistic regression model to the data
lr.fit(scaler.fit_transform(X_train), y_train)

# Calculates the accuracy on the test set and prints coefficients
acc = accuracy_score(y_test, lr.predict(scaler.transform(X_test)))
print(f"{acc:.1%} accuracy on test set.")
print(dict(zip(X.columns, abs(lr.coef_[0]).round(2))))

#### Automatic recursive feature elimination

Now let's automate this recursive process. Wrap a Recursive Feature Eliminator (RFE) around our logistic regression estimator and pass it the desired number of features.

All the necessary functions and packages have been pre-loaded and the features have been scaled for you.

##### Instructions

* Create the RFE with a `LogisticRegression()` estimator and 3 features to select.
* Print the features and their ranking.
* Print the features that are not eliminated.

In [ ]:
# setup
import pandas as pd

from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

diabetes_df = pd.read_csv("data/pima_indians_diabetes_2.csv")
# diabetes_df = diabetes_df.drop(["pregnant"], axis=1)

y = diabetes_df["test"]
X = diabetes_df.drop("test", axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

scaler = StandardScaler()
lr = LogisticRegression()

In [ ]:
rfe = RFE(estimator=LogisticRegression(max_iter=500, random_state=0), n_features_to_select=3, verbose=1)

rfe.fit(X_train, y_train)

print(dict(zip(X.columns, rfe.ranking_)))

print(X.columns[rfe.support_])

acc = accuracy_score(y_test, rfe.predict(X_test))
print(f"{acc:.1%} accuracy on test set.")

The site and I are getting different results. They consistently get:

```sh
{'pregnant': 5, 'glucose': 1, 'diastolic': 6, 'triceps': 3, 'insulin': 4, 'bmi': 1, 'family': 2, 'age': 1} 
Index(['glucose', 'bmi', 'age'], dtype='object')
80.6% accuracy on test set.
```

While my results vary but almost always include `'pregnant'` unless I drop it from the dataset.

According to my experiments below, the site and I are producing identical correlation matrices and our heatmaps are not surprisingly the same as well.

Interestingly, if I don't increase the `max_iter` parameter I get the following *after* my results:

```sh
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
```

The value I have to set for `max_iter` is not constant but I've never seen the error with a value >= 200. I am now wondering if the default solver has changed?

Checking the versions, on the site:

```sh
In [17]: import sklearn
In [18]: print('sklearn: {}'. format(sklearn. __version__))
sklearn: 1.0
```

and on my machine:

```sh
>>> import sklearn
>>> print('sklearn: {}'. format(sklearn. __version__))
sklearn: 1.3.2
```

After checking, however, (`print(LogisticRegression().solver)`) both the site and I are using `lbfgs`.

Next, I thought I would try installing scikit-learn v1.0 on my machine to see if I can reproduce the site's results. This, however, turned out to be more involved than I expected. Instead, I built a separate env with numpy v1.19.5, scikit-learn v1.0, and Python v3.9.7 to mirror the site's environment.



In [ ]:
print(LogisticRegression().solver)

In [ ]:
diabetes_df.corr(numeric_only=True)

In [ ]:
import seaborn as sns
import numpy as np

cmap = sns.diverging_palette(h_neg=10, h_pos=240, as_cmap=True)

corr = diabetes_df.corr(numeric_only=True)

mask = np.triu(np.ones_like(corr, dtype=bool))

sns.heatmap(corr, mask=mask, center=0, cmap=cmap, linewidths=1, annot=True, fmt=".2f")

Had to set `max_iter` to prevent an error. 

My results (`Index(['glucose', 'bmi', 'family'], dtype='object')`) are not matching the site's (`Index(['glucose', 'bmi', 'age'], dtype='object')`). 

I dropped the `'pregnant'` column because it was selected every time.

I think I must be doing something wrong, `'glucose'` is often not selected and `'age'` never is and if `'pregnant'` is not removed it is *always* selected.

## Tree-based feature selection

Some models will perform feature selection by design to avoid overfitting.

### Random forest classifier

One of those, is the random forest classifier - an ensemble model that will pass different, random, subsets of features to a number of decision trees. It then aggregates the predictions of the individual trees. The example forest shown here contains four decision trees.

![alt text](images/forest_of_decision_trees.png)

While simple in design, random forests often manage to be highly accurate and avoid overfitting even with the default Scikit-learn settings. 

Training a random forest classifier on the numeric features of the ANSUR dataset to predict gender, its test set accuracy is 99%. This means it managed to escape the curse of dimensionality and didn't overfit on the many features in the training set. 

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

ansur_df = pd.read_csv("data/ansur_gender_numeric.csv")

y = ansur_df["Gender"]
X = ansur_df.drop("Gender", axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)

print(f"{accuracy_score(y_test, rf.predict(X_test)):.2f}")

In this illustration of what the trained model could look like, the first decision tree in the forest used the neck circumference feature on its first decision node and hand length later on to determine if a person was male or female.

![alt text](images/ansur_gender_forest_of_decision_trees.png)

By averaging how often features are used to make decisions inside the different decision trees, and taking into account whether these are important decisions near the root of the tree or less important decisions in the smaller branches of the tree, the random forest algorithm manages to calculate feature importance values. 

### Feature importance values

These values can be extracted from a trained model using the `feature_importances_` attribute. 

In [ ]:
print(rf.feature_importances_)

Just like the coefficients produced by the logistic regressor, these feature importance values can be used to perform feature selection, since for unimportant features they will be close to zero. An advantage of these feature importance values over coefficients is that they are comparable between features by default, since they always sum up to one. Which means we don't have to scale our input data first. 

### Feature importance as a feature selector

We can use the feature importance values to create a `True`/`False` mask for features that meet a certain importance threshold. 

In [ ]:
mask = rf.feature_importances_ > 0.1

print(mask)

Then, we can apply that mask to our feature DataFrame to implement the actual feature selection. 

In [ ]:
X_reduced = X.loc[:, mask]

print(X_reduced.columns)

### RFE with random forests

Dropping one weak feature can make other features relatively more or less important.

To play it safe and minimize the risk of dropping the wrong features, you should not drop all least important features at once but rather one by one. To do so we can once again wrap a **Recursive Feature Eliminator**, or `RFE()`, around our model.

Here, we've reduced the number of features to six with no reduction in test set accuracy. However, training the model once for each feature we want to drop can result in too much computational overhead. 

To speed up the process we can pass the `step` parameter to `RFE()`. Here, we've set it to 10 so that on each iteration the 10 least important features are dropped. 

In [ ]:
from sklearn.feature_selection import RFE

y = ansur_df["Gender"]
X = ansur_df.drop("Gender", axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# rfe = RFE(estimator=RandomForestClassifier(random_state=0), n_features_to_select=6, verbose=1)
rfe = RFE(estimator=RandomForestClassifier(random_state=0), n_features_to_select=6, step=10, verbose=1)

rfe.fit(X_train, y_train)

# print(accuracy_score(y_test, rfe.predict(X_test)))
print(f"{accuracy_score(y_test, rfe.predict(X_test)):.2f}")

Once the final model is trained, we can use the feature eliminator's .support_ attribute as a mask to print the remaining column names. 

In [ ]:
print(X.columns[rfe.support_])

#### Building a random forest model

You'll again work on the Pima Indians dataset to predict whether an individual has diabetes. This time using a random forest classifier. You'll fit the model on the training data after performing the train-test split and consult the feature importance values.

The feature and target datasets have been pre-loaded for you as `X` and `y`. Same goes for the necessary packages and functions.

##### Instructions

* Set a 25% test size to perform a 75%-25% train-test split.
* Fit the random forest classifier to the training data.
* Calculate the accuracy on the test set.
* Print the feature importances per feature.

In [ ]:
# setup
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

ansur_df = pd.read_csv("data/pima_indians_diabetes_2.csv")

y = ansur_df["test"]
X = ansur_df.drop("test", axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)

acc = accuracy_score(y_test, rf.predict(X_test))

print(dict(zip(X.columns, rf.feature_importances_.round(2))))

print(f"{acc:.1%} accuracy on test set.")

#### random forest for feature selection

Now lets use the fitted random model to select the most important features from our input dataset `X`.

The trained model from the previous exercise has been pre-loaded for you as `rf`.

##### Instructions 1/2

* Create a mask for features with an importance higher than 0.15

In [ ]:
mask = rf.feature_importances_ > 0.15

print(mask)

##### Instructions 2/2

* Sub-select the most important features by applying the mask to `X`

In [ ]:
reduced_X = X.loc[:, mask]

print(reduced_X.columns)

#### Recursive feature elimination with random forests 

You'll wrap a Recursive Feature Eliminator around a random forest model to remove features step by step. This method is more conservative compared to selecting features after applying a single importance threshold. Since dropping one feature can influence the relative importances of the others.

You'll need these pre-loaded datasets: `X`, `X_train`, `y_train`.

Functions and classes that have been pre-loaded for you are: `RandomForestClassifier()`, `RFE()`, `train_test_split()`.

##### Instructions 1/4

* Create a recursive feature eliminator that will select the 2 most important features using a random forest model.

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

# from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

ansur_df = pd.read_csv("data/pima_indians_diabetes_2.csv")

y = ansur_df["test"]
X = ansur_df.drop("test", axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
rfe = RFE(estimator=RandomForestClassifier(random_state=0), n_features_to_select=2, verbose=1)

##### Instructions 2/4

* Fit the recursive feature eliminator to the training data.

In [ ]:
rfe.fit(X_train, y_train)

##### Instructions 3/4

* Create a mask using the fitted eliminator's `support_` attribute, then apply it to the feature dataset `X`.

In [ ]:
mask = rfe.support_

reduced_X = X.loc[:, mask]
print(reduced_X.columns)

##### Instructions 4/4

* Change the settings of `RFE()` to eliminate 2 features at each step.

In [ ]:
rfe = RFE(estimator=RandomForestClassifier(), n_features_to_select=2, step=2, verbose=1)

rfe.fit(X_train, y_train)

mask = rfe.support_

reduced_X = X.loc[:, mask]
print(reduced_X.columns)

## Regularized linear regression

So far, we focused on how to reduce dimensionality using classification algorithms. Let's see what we can do with regressions. 

### Linear model concept

To refresh how linear regressions work, we'll build a model that derives the linear function between three input values and a target. 

However, we'll be creating the feature dataset and linear function ourselves, so that we can control the ground truth that our model tries to derive. 

### Creating our own dataset

Create three features x1, x2, and x3 that all follow a simple normal distribution.

![alt text](images/dataset_distribution.png)

We can then create a target y with a function of our choice.

$$
y = 20 + 5x_1 + 2x_2 + 0x_3 + error
$$

The $20$ is called the intercept.

The $5$, $2$, and $0$ are the coefficients of our features and determine how large an effect each has on the target.

Since the third feature has a coefficient of 0 and will have no effect on the target it would be best to remove it so it does not confuse the model and cause overfitting.

### Linear regression in Python

When you fit a `LinearRegression()` model with Scikit Learn, the model object will have `.coef_` for coefficient attribute that contains a NumPy array with a number of elements equal to the number of input features. These are the three values we just set to 5, 2, and 0 and the model was able to estimate them pretty accurately.

```python
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)

print(lr.coef_)
```

```sh
[ 4.95 1.83 -0.05]
```

Same goes for the intercept. 

```python
print(lr.intercept_)
```

```sh
19.8
```

To check how accurate the model's predictions are we can calculate the R-squared value on the test set. This tells us how much of the variance in the target feature our model can predict. 

```python
print(lr.score(X_test, y_test))
```

```sh
0.976
```

Our model scores an impressive 97.6%. 

However, the third feature, which had no effect whatsoever, was estimated to have a small effect of -0.05. If there would be more of these irrelevant features, the model could overfit. To solve this, we'll have to look at what the model actually does while training. 

### Loss function: Mean Squared Error

The model will try to find optimal values for the intercept and coefficients by minimizing a loss function. 

This function contains the mean sum of the squared differences between actual and predicted values, the gray squares in the plot.

![alt text](images/mse.png)

Minimizing this MSE makes the model as accurate a possible. However, we don't want our model to be super accurate on the training set if that means it no longer generalizes to new data. 

### Adding regularization

To avoid this we can introduce regularization. The model will then not only try to be as accurate as possible by minimizing the MSE, it will also try to keep the model simple by keeping the coefficients low. The strength of regularization can be tweaked with alpha, when it's too low the model might overfit, when it's too high the model might become too simple and inaccurate.

$$\alpha(\lvert \Beta_1 \rvert + \lvert \Beta_2 \rvert + \lvert \Beta_3 \rvert)$$

![alt text](images/regularization.png)

One linear model that includes this type of regularization is called Lasso, for least absolute shrinkage and selection operator. 

### Lasso regressor

When we fit it on our dataset we see that it indeed reduced the coefficient of the third feature to zero, ignoring it, but also that it reduced the other coefficients resulting in a lower R squared. 

```python
from sklearn.linear_model import Lasso

la = Lasso()
la.fit(X_train, y_train)

print(la.coef_)
```

```sh
[4.07 0.59 0. ]
```

```python
print(la.score(X_test, y_test))
```

```sh
0.861
```

To avoid this we can change the alpha parameter. When we set it to 0.05 the third feature is still ignored but the other coefficients are reduced less and our R squared is up again. 

```python
from sklearn.linear_model import lasso

la = lasso(alpha=0.05)
la.fit(X_train, y_train)

print(la.coef_)
```

```sh
[4.91 1.76 0. ]
```

```python
print(la.score(X_test, y_test))
```

```sh
0.974
```

#### Creating a LASSO regressor

You'll be working on the numeric ANSUR body measurements dataset to predict a persons Body Mass Index (BMI) using the pre-imported `Lasso()` regressor. BMI is a metric derived from body height and weight but those two features have been removed from the dataset to give the model a challenge.

You'll standardize the data first using the `StandardScaler()` that has been instantiated for you as `scaler` to make sure all coefficients face a comparable regularizing force trying to bring them down.

All necessary functions and classes plus the input datasets `X` and `y` have been pre-loaded.

##### Instructions

* Set the test size to 30% to get a 70-30% train test split.
* Fit the scaler on the training features and transform these in one go.
* Create the Lasso model.
* Fit it to the scaled training data.

In [ ]:
# setup
import pandas as pd

from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

ansur_df = pd.read_csv("data/ansur_numeric.csv")

y = ansur_df["BMI"]
X = ansur_df.drop("BMI", axis=1)

scaler = StandardScaler()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

X_train_std = scaler.fit_transform(X_train, y_train)

la = Lasso()

la.fit(X_train_std, y_train)

#### Lasso model results

Now that you've trained the Lasso model, you'll score its predictive capacity ($R^2$) on the test set and count how many features are ignored because their coefficient is reduced to zero.

The `X_test` and `y_test` datasets have been pre-loaded for you.

The `Lasso()` model and `StandardScaler()` have been instantiated as `la` and `scaler` respectively and both were fitted to the training data.

##### Instructions

* Transform the test set with the pre-fitted scaler.
* Calculate the $R^2$ value on the scaled test data.
* Create a list that has True values when coefficients equal 0.
* Calculate the total number of features with a coefficient of 0.

In [ ]:
# setup
import pandas as pd

from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

ansur_df = pd.read_csv("data/ansur_numeric.csv")

y = ansur_df["BMI"]
X = ansur_df.drop("BMI", axis=1)

scaler = StandardScaler()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

X_train_std = scaler.fit_transform(X_train)

la = Lasso()

la.fit(X_train_std, y_train)

In [ ]:
# transform the test set with the pre-fitted scaler
X_test_std = scaler.transform(X_test)

# calculate the coefficient of determination (R squared) on X_test_std
r_squared = la.score(X_test_std, y_test)
print(f"The model can predict {r_squared:.1%} of the variance in the test set.")

# Create a list that has True values when coefficients equal 0
zero_coef = la.coef_ == 0

# Calculate how many features have a zero coefficient
n_ignored = sum(zero_coef)
print(f"The model has ignored {n_ignored} out of {len(la.coef_)} features.")

#### Adjusting the regularization strength

Your current Lasso model has an $R^2$ score of 84.7%. When a model applies overly powerful regularization it can suffer from high bias, hurting its predictive power.

Let's improve the balance between predictive power and model simplicity by tweaking the `alpha` parameter.

##### Instructions

* Find the highest value for `alpha` that gives an $R^2$ value above 98% from the options: `1`, `0.5`, `0.1`, and `0.01`.

In [ ]:
# Find the highest alpha value with R-squared above 98%
# la = Lasso(alpha=1, random_state=0)
# la = Lasso(alpha=0.5, random_state=0)
la = Lasso(alpha=0.1, random_state=0)
# la = Lasso(alpha=0.01, random_state=0)

# Fits the model and calculates performance stats
la.fit(X_train_std, y_train)
r_squared = la.score(X_test_std, y_test)
n_ignored_features = sum(la.coef_ == 0)

# Print peformance stats
print(f"The model can predict {r_squared:.1%} of the variance in the test set.")
print(f"{n_ignored_features} out of {len(la.coef_)} features were ignored.")

## Combining feature selectors

In the previous lesson we saw how Lasso models allow you to tweak the strength of regularization with the alpha parameter. 

We manually set this alpha parameter to find a balance between removing as much features as possible and model accuracy. However, manually finding a good alpha value can be tedious. Good news is, there is a way to automate this. 

### LassoCV regressor

The `LassoCV()` class will use cross validation to try out different alpha settings and select the best one. When we fit this model to our training data it will get an `alpha_` attribute with the optimal value. 

```python
from sklearn.linear_model import LassoCV

lcv = LassoCV()
lcv.fit(X_train, y_train)

print(lcv.alpha_)
```

```sh
0.09
```

To actually remove the features to which the Lasso regressor assigned a zero coefficient, we once again create a mask with True values for all non-zero coefficients. We can then apply it to our feature dataset `X` with the `loc` method. 

```python
mask = lcv.coef_ != 0

reduced_X = X.loc[:, mask]
```

### Taking a step back

* Random forest is a combination of decision trees (multiple weak predictors can combine to form a strong one)
* We can use a combination of models for feature selection too

### Feature selection with LassoCV

To do so lets first train the models one by one.

We'll be predicting BMI in the ANSUR dataset just like you did in the last exercises. If we use `LassoCV()` we'll get an R squared of 99% and when we create a mask that tells us whether a feature has a non-zero coefficient we see that this is true for 66 out of 91 features. We'll put this `lcv_mask` to the side for a moment and move on to the next model.

```python
from sklearn.linear_model import LassoCV

lcv-LassoCV()
lcv.fit(X_train, y_train)
lcv.score(X_test, y_test)
```

```sh
0.99
```

```python
lcv_mask = lcv.coef_ != 0
sum(lcv_mask)
```

```sh
66
```

### Feature selection with random forest

The second model we train is a random forest regressor model. We've wrapped a Recursive Feature Selector or RFE, around the model to have it select the same number of features as the `LassoCV()` regressor did. We can then use the `support_` attribute of the fitted model to create `rf_mask`. 

```python
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor

rfe_rf = RFE(estimator=RandomForestRegressor(), n_features_to_select=66, step=5, verbose=1)
rfe_fr.fit(X_train, y_train)
rf_mask = rfe_rf.support_
```

### Feature selection with gradient boosting

Then, we do the same thing with a gradient boosting regressor. Like random forests gradient boosting is an ensemble method that will calculate feature importance values. The trained model too has a `support_` attribute that we can use to create `gb_mask`. 

```python
from sklearn.feature_selection import RFE
from sklearn.ensemble import GradientBoostingRegressor

rfe_gb = RFE(estimator=GradientBoostingRegressor(), n_features_to_select=66, step=5, verbose=1)
rfe_gb.fit(X_train, y_train)
gb_mask = rfe_gb.support_
```

### Combining the feature selectors

Finally, we can start counting the votes on whether to select a feature. We use NumPy's `sum()` function, pass it the three masks in a list, and set the `axis` argument to 0. 
```python
import numpy as np

votes = np.sum([lcv_mask, rf_mask, gb_mask], axis=0)
print(votes)
```

We'll then get an array with the number of votes that each feature got.

```sh
array([3, 2, 2, ..., 3, 0, 1])
```



What we do with this vote depends on how conservative we want to be.

If we want to make sure we don't lose any information, we could select all features with at least one vote.

In this example we chose to have at least two models voting for a feature in order to keep it.

```python
mask = votes > 2
```

All that is left now is to actually implement the dimensionality reduction. We do that with the `loc` method of our feature DataFrame `X`. 

```python
reduced_x = X.loc[:, mask]
```

#### Creating a LassoCV regressor

You'll be predicting biceps circumference on a subsample of the male ANSUR dataset using the `LassoCV()` regressor that automatically tunes the regularization strength (alpha value) using Cross-Validation.

The standardized training and test data has been pre-loaded for you as `X_train`, `X_test`, `y_train`, and `y_test`.

##### Instructions

* Create and fit the LassoCV model on the training set.
* Calculate $R^2$ on the test set.
* Create a mask for coefficients not equal to zero.

In [13]:
# setup
import pandas as pd

from sklearn.linear_model import Lasso

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV

ansur_df = pd.read_csv("data/ansur_lasso.csv")

y = ansur_df["biceps circumference flexed"]
X = ansur_df.drop("biceps circumference flexed", axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [14]:
# create and fit the LassoCV model on the training set
lcv = LassoCV(random_state=0)
lcv.fit(X_train, y_train)
print(f"Optimal alpha = {lcv.alpha_:.3f}")

# calculate R squared on the test set
r_squared = lcv.score(X_test, y_test)
print(f"The model explains {r_squared:.1%} of the test set variance")

# create a mask for coefficients not equal to zero
lcv_mask = lcv.coef_ != 0
print(f"{sum(lcv_mask)} features out of {len(lcv_mask)} selected")

Optimal alpha = 2.171
The model explains 88.1% of the test set variance
20 features out of 32 selected


In [15]:
selected_features = sorted(X_train.columns[lcv_mask])

print("Selected features:")
for feature in selected_features:
    print(feature)

Selected features:
acromialheight
bideltoidbreadth
buttockcircumference
buttockkneelength
buttockpopliteallength
chestcircumference
chestheight
forearmcircumferenceflexed
iliocristaleheight
interscyeii
lateralfemoralepicondyleheight
lateralmalleolusheight
radialestylionlength
shouldercircumference
shoulderelbowlength
thighcircumference
thighclearance
verticaltrunkcircumferenceusa
waistcircumference
waistdepth


#### Ensemble models for extra votes

The `LassoCV()` model selected 22 out of 32 features. Not bad, but not a spectacular dimensionality reduction either. Let's use two more models to select the 10 features they consider most important using the Recursive Feature Eliminator (RFE).

The standardized training and test data has been pre-loaded for you as `X_train`, `X_test`, `y_train`, and `y_test`.


##### Instructions 1/4

* Select 10 features with RFE on a `GradientBoostingRegressor` and drop 3 features on each step.

In [16]:
# setup
import pandas as pd

from sklearn.feature_selection import RFE
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV

ansur_df = pd.read_csv("data/ansur_lasso.csv")

y = ansur_df["biceps circumference flexed"]
X = ansur_df.drop("biceps circumference flexed", axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [17]:
rfe_gb = RFE(estimator=GradientBoostingRegressor(), n_features_to_select=10, step=3, verbose=1)
rfe_gb.fit(X_train, y_train)

Fitting estimator with 32 features.
Fitting estimator with 29 features.
Fitting estimator with 26 features.
Fitting estimator with 23 features.
Fitting estimator with 20 features.
Fitting estimator with 17 features.
Fitting estimator with 14 features.
Fitting estimator with 11 features.


RFE(estimator=GradientBoostingRegressor(), n_features_to_select=10, step=3,
    verbose=1)

##### Instructions 2/4

* Calculate the $R^2$ on the test set.

In [18]:
r_squared = rfe_gb.score(X_test, y_test)
print(f"The model can explain {r_squared:.1%} of the variance in the test set")

The model can explain 85.6% of the variance in the test set


##### Instructions 3/4

* Assign the support array of the fitted model to `gb_mask`.

In [19]:
gb_mask = rfe_gb.support_

##### Instructions 4/4

* Modify the first step to select 10 features with RFE on a `RandomForestRegressor()` and drop 3 features on each step.

In [20]:
# select 10 features with RFE on a RandomForestRegressor, drop 3 features on each step
rfe_rf = RFE(estimator=RandomForestRegressor(), n_features_to_select=10, step=3, verbose=1)
rfe_rf.fit(X_train, y_train)

# Calculate the R squared on the test set
r_squared = rfe_rf.score(X_test, y_test)
print(f"The model can explain {r_squared:.1%} of the variance in the test set")

# Assign the support array to rf_mask
rf_mask = rfe_rf.support_

Fitting estimator with 32 features.
Fitting estimator with 29 features.
Fitting estimator with 26 features.
Fitting estimator with 23 features.
Fitting estimator with 20 features.
Fitting estimator with 17 features.
Fitting estimator with 14 features.
Fitting estimator with 11 features.
The model can explain 83.9% of the variance in the test set


#### Combining 3 feature selectors

We'll combine the votes of the 3 models you built in the previous exercises, to decide which features are important into a meta mask. We'll then use this mask to reduce dimensionality and see how a simple linear regressor performs on the reduced dataset.

The per model votes have been pre-loaded as `lcv_mask`, `rf_mask`, and `gb_mask` and the feature and target datasets as X and y.

##### Instructions 1/4

* Sum the votes of the three models using `np.sum()`.

In [22]:
# setup
import numpy as np

In [24]:
votes = np.sum([lcv_mask, rf_mask, gb_mask], axis=0)
print(votes)

[1 0 3 3 1 1 0 3 1 0 0 3 0 0 0 1 1 1 1 2 0 1 3 1 0 3 2 2 2 1 1 2]


##### Instructions 2/4

* Create a mask for features selected by all 3 models.

In [25]:
meta_mask = votes == 3
print(meta_mask)

[False False  True  True False False False  True False False False  True
 False False False False False False False False False False  True False
 False  True False False False False False False]


##### Instructions 3/4

* Apply the dimensionality reduction on X and print which features were selected.

In [28]:
X_reduced = X.loc[:, meta_mask]
print(sorted(X_reduced.columns))

['bideltoidbreadth', 'buttockcircumference', 'chestcircumference', 'forearmcircumferenceflexed', 'shouldercircumference', 'thighcircumference']


##### Instructions 4/4

* Plug the reduced dataset into the code for simple linear regression that has been written for you.

In [31]:
# setup
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

lm = LinearRegression()
scaler = StandardScaler()

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.3, random_state=0)
lm.fit(scaler.fit_transform(X_train), y_train)
r_squared = lm.score(scaler.transform(X_test), y_test)
print(f"The model can explain {r_squared:.1%} of the variance in the test set using {len(lm.coef_)} features.")

The model can explain 86.6% of the variance in the test set using 6 features.
